In [11]:
%pwd

'c:\\Users\\Dzuels Foundation\\mlproject\\notebook'

In [12]:
%cd ..\


c:\Users\Dzuels Foundation\mlproject


In [8]:
import requests
import os
import sys
import pandas as pd
from pathlib import Path
from dataclasses import dataclass

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from joblib import dump

from src.constant import *
from src.logger import logging
from src.exception import CustomException
from src.utils import read_yaml,create_directory


In [4]:
@dataclass(frozen=True)
class ModelTrainingConfig:
    """
    Data Model Training Configuration
    """

    root_dir: Path
    train_data_file: Path
    model_file: Path
    all_params: dict
    target_column: str
    

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path=CONFIG_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)
                
        create_directory(self.config.artifact_root)
        
    def get_model_trainer_config(self) -> ModelTrainingConfig:
        """
        Returns the Model Training Configuration
        """
        root_dir = self.config.model_training.root_dir
        all_params = self.params.PARAMETERS.model.params
        target_column = self.schema.TARGET_COLUMN.name
        train_data_file = self.config.model_training.train_data_file
        model_file = self.config.model_training.model_file
        
        
        create_directory(self.config.model_training.root_dir)
        
        model_training_config = ModelTrainingConfig(
            root_dir=root_dir,
            train_data_file=train_data_file,
            model_file=model_file,
            all_params=all_params,
            target_column=target_column
        )
        
        return model_training_config
        
        
        
        

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
        
    def train_model(self):
        try:
            logging.info("Loading data")
            df = pd.read_csv(self.config.train_data_file)
            
            X = df.drop(columns=[self.config.target_column])
            y = df[self.config.target_column]
            model = ElasticNet(
                alpha=self.config.all_params.alpha,
                l1_ratio=self.config.all_params.l1_ratio,
                random_state=42
            )
            model.fit(X, y)
            logging.info("Model training completed")
            
            logging.info("Saving the trained model")
            # Save the model to the specified path
            dump(model, self.config.model_file)
            logging.info(f"Model saved at {self.config.model_file}")
            
        except Exception as e:
            raise CustomException(e, sys)

In [13]:
try:
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train_model()
except Exception as e:
    logging.error(f"Error occurred during model training: {str(e)}")
    raise CustomException(e, sys)

[2025-04-08 13:30:24,593] : root : INFO: yaml file: config\config.yaml loaded successfully
[2025-04-08 13:30:24,734] : root : INFO: yaml file: schema.yaml loaded successfully
[2025-04-08 13:30:24,736] : root : INFO: yaml file: params.yaml loaded successfully
[2025-04-08 13:30:24,761] : root : INFO: Directory 'artifact' already exists.
[2025-04-08 13:30:24,762] : root : INFO: Directory 'artifact/model_training' created.
[2025-04-08 13:30:24,762] : root : INFO: Loading data
[2025-04-08 13:30:25,410] : root : INFO: Model training completed
[2025-04-08 13:30:25,411] : root : INFO: Saving the trained model
